# Using Partitioning to Optimize a Circuit

Synthesis is a very powerful circuit optimization technique. However, the input size to even QFAST doesn't scale to larger circuits well. In fact, to be able to synthesize a circuit currently, we will need to be able to simulate it. This will ultimately cap the scaling of synthesis algorithms. However, we can still use a synthesis tool together with partitioner to optimize small blocks of a circuit at a time. BQSKit was designed for this exact use case and in this guide, we will explore how to accomplish this. It is recommended that you read the other tutorials first.

In [1]:
# Load a 16-qubit time evolution circuit generated from the ArQTIC circuit generator.
from bqskit.ir import Circuit

circuit = Circuit.from_file('tfim-16-20.qasm')

for gate in circuit.gate_set:
    print(f"{gate} Count:", circuit.count(gate))

RXGate Count: 320
CNOTGate Count: 600
RZGate Count: 300


We will partition the circuit and then use the `ForEachBlockPass` to perform operations on the individual blocks. Note the `ForEachBlockPass` will run the sub tasks in parallel using dask.

In [3]:
from bqskit.compiler import CompilationTask
from bqskit.compiler import Compiler
from bqskit.passes import QuickPartitioner
from bqskit.passes import ForEachBlockPass
from bqskit.passes import QSearchSynthesisPass
from bqskit.passes import ScanningGateRemovalPass
from bqskit.passes import UnfoldPass

task = CompilationTask(circuit, [
    QuickPartitioner(3),
    ForEachBlockPass([QSearchSynthesisPass(), ScanningGateRemovalPass()]),
    UnfoldPass(),
])

# Finally, we construct a compiler and submit the task
with Compiler() as compiler:
    synthesized_circuit = compiler.compile(task)
    
for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))

U3Gate 8761534630262
{U3Gate: 11, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 11, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 11, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 10, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 9, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 10, CNOTGate: 4}
CNOTGate 8761534630352
{U3Gate: 9, CNOTGate: 4}
CNOTGate 8761534630352
{U3Gate: 8, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 9, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 8, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 8, CNOTGate: 4}
CNOTGate 8761534630352
{U3Gate: 8, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 7, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 8, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 8, CNOTGate: 4}
CNOTGate 8761534630352
{U3Gate: 7, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 7, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 6, CNOTGate: 4}
CNOTGate 8761534630352
{U3Gate: 6, CNOTGate: 4}
CNOTGate 8761534630352
{U3Gate: 7, CNOTGate: 4}
U3Gate 8761534630262
{U3Gate: 6, CNOTGate: 4}
U3Gate 8761534630

U3Gate 8749012685439U3Gate 8745777702798
{U3Gate: 11, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 11, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 10, CNOTGate: 4}
CNOTGate 8745777702711
{U3Gate: 9, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 9, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 8, CNOTGate: 4}
CNOTGate 8745777702711
{U3Gate: 8, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 8, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 8, CNOTGate: 4}
CNOTGate 8745777702711
{U3Gate: 7, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 7, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 7, CNOTGate: 4}
CNOTGate 8745777702711
{U3Gate: 7, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 7, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 7, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 11, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 10, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 9, CNOTGate: 4}
CNOTGate 8745777702711
{U3Gate: 8, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 8, CNOTGate: 4}
U3Gate 8745777702798
{U3Gate: 7, CNOTGate: 4}

U3Gate 8745777702798U3Gate 8790059379478
{U3Gate: 11, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 10, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 9, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 8, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 8, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 7, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 7, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 7, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 7, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 6, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 6, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 11, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 10, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 9, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 8, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 8, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 7, CNOTGate: 4}



{U3Gate: 7, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 7, CNOTGate: 4}
CNOTGate 8749012685376
{U3Gate: 6, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 6, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 5, CNOTGate: 4}
CNOTGate 8749012685376
{U3Gate: 5, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 5, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 5, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 11, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 10, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 9, CNOTGate: 4}
CNOTGate 8749012685376
{U3Gate: 8, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 8, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 7, CNOTGate: 4}
CNOTGate 8749012685376
{U3Gate: 7, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 7, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 7, CNOTGate: 4}
CNOTGate 8749012685376
{U3Gate: 6, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 6, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 5, CNOTGate: 4}
CNOTGate 8749012685376
{U3Gate: 5, CNOTGate: 4}
U3Gate 8749012685439
{U3Gate: 5, CNOTGat

{U3Gate: 6, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 11, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 10, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 9, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 8, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 8, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 7, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 7, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 7, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 6, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
CNOTGate 8790059379484
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 5, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 11, CNOTGate: 4}
U3Gate 8790059379478
{U3Gate: 10, CNOTGate

...

{CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate

{CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1,

{CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate

{CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(0,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(1, 2) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[CNOTGate@(0, 1) ... U3Gate@(1,)]): 1, U3Gate: 635, CNOTGate: 510}
Cir

CNOTGate Count: 576
U3Gate Count: 717


## Gatesets

Just like we changed the gates used by QSearch in the Search Synthesis tutorial, we can change the gates for the entire circuit using the same method.

**Exercise:** Change the gates used in the below example to change the gate set for the circuit.

In [5]:
from bqskit.ir.gates import ISwapGate, U3Gate
from bqskit.passes.search import SimpleLayerGenerator

layer_gen = SimpleLayerGenerator(two_qudit_gate=ISwapGate(), single_qudit_gate_1=U3Gate())

configured_qsearch_pass = QSearchSynthesisPass(layer_generator=layer_gen)

task = CompilationTask(circuit, [
    QuickPartitioner(3),
    ForEachBlockPass([configured_qsearch_pass, ScanningGateRemovalPass()]),
    UnfoldPass(),
])

with Compiler() as compiler:
    synthesized_circuit = compiler.compile(task)
    
for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))

U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 9, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 9, ISwapGate: 4}
U3Gate 8749521503654
{U3Gat

U3Gate 8727527363389
{U3Gate: 11, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 10, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 10, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 9, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 9, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 9, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 8, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 7, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 7, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 7, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 7, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 11, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 10, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 9, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 9, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 9, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 9, ISwapGa

{U3Gate: 9, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 8, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 11, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 10, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 7, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 10, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 7, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 7, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 9, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 7, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 9, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 9, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 8, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 8, ISwapGate: 4}
ISwapGate 8727527188917
{U3Gate: 7, ISwapGate: 4}
U3Gate 8727527363389
{U3Gate: 7, ISwapGate: 4}
U3Gate 872752

U3Gate 8749521503654
{U3Gate: 7, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 9, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 9, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 9, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 8, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 8, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 8, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 8, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 8, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 7, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 7, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 7, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 7, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 9, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 9, ISwapG

{U3Gate: 13, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 12, ISwapGate: 6}
ISwapGate 8749548856387
{U3Gate: 12, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 12, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 12, ISwapGate: 6}
ISwapGate 8749548856387
{U3Gate: 12, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 12, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 12, ISwapGate: 6}
ISwapGate 8749548856387
{U3Gate: 12, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 12, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 12, ISwapGate: 6}
ISwapGate 8749548856387
{U3Gate: 12, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 12, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 11, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 11, ISwapGate: 6}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate: 4}
ISwapGate 8749548856387
{U3Gate: 10, ISwapGate: 4}
U3Gate 8749521503654
{U3Gate: 10, ISwapGate

...

{CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@

{CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@

{CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@

{CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(0,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, CircuitGate(Circuit(3)[U3Gate@(1,) ... U3Gate@(2,)]): 1, CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]): 1, U3Gate: 829, ISwapGate: 510}
CircuitGate(Circuit(3)[U3Gate@(2,) ... U3Gate@(1,)]) 19134

ISwapGate Count: 578
U3Gate Count: 943


## Block Size

Increasing the partitioner's block size will likely lead to better results at a runtime cost. If you have the computing resources, you can launch a Dask cluster and connect to it via `Compiler()`. The ForEachBlockPass will efficiently distribute the work.

In [ ]:
from bqskit.passes import OptimizedLEAPPass


task = CompilationTask(circuit, [
    QuickPartitioner(4),
    ForEachBlockPass([OptimizedLEAPPass]),
    UnfoldPass(),
])

with Compiler() as compiler:
    synthesized_circuit = compiler.compile(task)
    
for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))